# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

добывающая компания «ГлавРосГосНефть» предоставила данные геологоразведки из трех регионов для того, чтобы оценить прибыль и риски. задача проекта сравнить показатели возможной прибыли из трех регионов.

In [1]:
# pip install pandas-profiling

In [2]:
import pandas as pd
# from pandas_profiling import ProfileReport 
import numpy as np
import scipy.stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error 

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [5]:
# ProfileReport(geo_data_0)

In [6]:
# ProfileReport(geo_data_1)


In [7]:
# ProfileReport(geo_data_2)

Заметим высокую положительную корреляцию параметра f2 и дебитом скважины во всех регионах. 

In [8]:
geo_data_0.id.duplicated().sum()

10

In [9]:
geo_data_0 = geo_data_0.drop_duplicates(subset=['id'])

In [10]:
geo_data_0.shape

(99990, 5)

In [11]:
geo_data_1.id.duplicated().sum()

4

In [12]:
geo_data_1 = geo_data_1.drop_duplicates(subset=['id'])

In [13]:
geo_data_1.shape

(99996, 5)

In [14]:
geo_data_2.id.duplicated().sum()

4

In [15]:
geo_data_2 = geo_data_2.drop_duplicates(subset=['id'])

In [16]:
geo_data_2.shape

(99996, 5)

In [17]:
geo_data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [18]:
geo_data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [19]:
geo_data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


проверим пропуски в данных

In [20]:
geo_data_0.isna().sum()


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [21]:
geo_data_1.isna().sum()


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [22]:
geo_data_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [23]:
geo_data_0.duplicated().sum()

0

In [24]:
geo_data_1.duplicated().sum()

0

In [25]:
geo_data_2.duplicated().sum()

0

In [26]:
geo_data_0.describe()

,f0,f1,f2,product
count,99990.000000,99990.000000,99990.000000,99990.000000
mean,0.500454,0.250141,2.502629,92.499684
std,0.871844,0.504430,3.248149,44.288304
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072572,-0.200877,0.287784,56.497069
50%,0.502405,0.250252,2.515969,91.847928
75%,1.073626,0.700640,4.715035,128.563699
max,2.362331,1.343769,16.003790,185.364347


In [27]:
geo_data_1.describe()

,f0,f1,f2,product
count,99996.000000,99996.000000,99996.000000,99996.000000
mean,1.141209,-4.796608,2.494501,68.823916
std,8.965815,5.119906,1.703579,45.944663
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011475,57.085625
75%,8.620964,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [28]:
geo_data_2.describe()

,f0,f1,f2,product
count,99996.000000,99996.000000,99996.000000,99996.000000
mean,0.002002,-0.002159,2.495084,94.998342
std,1.732052,1.730397,3.473482,44.749573
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162328,-1.174841,0.130269,59.450028
50%,0.009424,-0.009661,2.484236,94.925026
75%,1.158477,1.163523,4.858720,130.586815
max,7.238262,7.844801,16.739402,190.029838


In [29]:
geo_data_0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [30]:
geo_data_0['product'].sum()


9249043.424932176

In [31]:
geo_data_1['product'].sum()


6882116.296140391

In [32]:
geo_data_2['product'].sum()

9499454.218564902

самый большой запас в скважине и суммарный запас  в третьем регионе

познокомившись с данными, удалил дубликаты скважин.

## Обучение и проверка модели

разобьем данные на треннировочные и валидационные

In [33]:
features_0 = geo_data_0.drop(['product','id'], axis=1)
target_0 = geo_data_0['product']

In [34]:
features_1 = geo_data_1.drop(['product','id'], axis=1)
target_1 = geo_data_1['product']

In [35]:
features_2 = geo_data_2.drop(['product','id'], axis=1)
target_2 = geo_data_2['product']

In [36]:
features_0_train,\
features_0_valid,\
target_0_train, \
target_0_valid = train_test_split(features_0,target_0,test_size=0.25, random_state=12345)

In [37]:
features_1_train,\
features_1_valid,\
target_1_train, \
target_1_valid = train_test_split(features_1,target_1,test_size=0.25, random_state=12345)

In [38]:
features_2_train,\
features_2_valid,\
target_2_train, \
target_2_valid = train_test_split(features_2,target_2,test_size=0.25, random_state=12345)

построим модель линейной регрессии для первого участка

In [39]:
model_0 = LinearRegression().fit(features_0_train,target_0_train)
predicted_valid_0 = model_0.predict(features_0_valid)
mean_absolute_error_0 = mean_absolute_error(target_0_valid, predicted_valid_0)
print('средняя абсолютная ошибка',mean_absolute_error_0)
mse_0 = mean_squared_error(target_0_valid, predicted_valid_0)
rmse_0 = mse_0 ** 0.5 
print('rmse:',rmse_0)
print('средний запас предсказанного сырья:',predicted_valid_0.mean())

средняя абсолютная ошибка 31.141028675220266
rmse: 37.853527328872964
средний запас предсказанного сырья: 92.78915638280621


построим модель линейной регрессии для второго участка

In [40]:
model_1 = LinearRegression().fit(features_1_train,target_1_train)
predicted_valid_1 = model_1.predict(features_1_valid)
mean_absolute_error_1 = mean_absolute_error(target_1_valid, predicted_valid_1)
print('средняя абсолютная ошибка',mean_absolute_error_1)
mse_1 = mean_squared_error(target_1_valid, predicted_valid_1)
rmse_1 = mse_1 ** 0.5 
print('rmse:',rmse_1)
print('средний запас предсказанного сырья:',predicted_valid_1.mean())

средняя абсолютная ошибка 0.7193530096516099
rmse: 0.892059264771703
средний запас предсказанного сырья: 69.17831957030432


построим модель линейной регрессии для третьего участка

In [41]:
model_2 = LinearRegression().fit(features_2_train,target_2_train)
predicted_valid_2 = model_2.predict(features_2_valid)
mean_absolute_error_2 = mean_absolute_error(target_2_valid, predicted_valid_2)
print('средняя абсолютная ошибка',mean_absolute_error_2)
mse_2 = mean_squared_error(target_2_valid, predicted_valid_2)
rmse_2 = mse_2 ** 0.5 
print('rmse:',rmse_2)
print('средний запас предсказанного сырья:',predicted_valid_2.mean())

средняя абсолютная ошибка 32.83139014902301
rmse: 40.07585073246016
средний запас предсказанного сырья: 94.86572480562035


самый высокий средний запас предсказанного сырья в третьем регионе. самая низкая средняя абсолютная ошибка в первом регионе, Модель во втором регионе-самая точная, у нее маленькая средняя адсолютная ошибка и маленький показатель rmse.

## Подготовка к расчёту прибыли

In [42]:
print('минимальный объем для окупаемости',round(10000000000/450000,2))

минимальный объем для окупаемости 22222.22


In [43]:
min_product = 10000000000/(450000*200) # минимальный объем  одной скважины для бузубыточной добычи

сравним это значение со средними показателями добычи в регионах

In [44]:
geo_data_0['product'].mean() > min_product

False

In [45]:
geo_data_1['product'].mean() > min_product

False

In [46]:
geo_data_2['product'].mean() > min_product

False

среднее значение оказалось меньше,Это может говорить о том, что мы не можем рандомно выбирать скважины для разработки, ведь в таком случае, нас, возможно, ждет неудача

In [47]:
largest_200_geo_0 = geo_data_0.sample(500,random_state=12345).sort_values('product',ascending=False).head(200)

In [48]:
largest_200_geo_1 = geo_data_1.sample(500,random_state=12345).sort_values('product',ascending=False).head(200)

In [49]:
largest_200_geo_2 = geo_data_2.sample(500,random_state=12345).sort_values('product',ascending=False).head(200)

In [50]:
largest_200_geo_0['product'].head()

80081    185.237672
8481     184.503922
70364    183.731356
69145    183.043169
75171    178.754659
Name: product, dtype: float64

In [51]:
largest_200_geo_1['product'].head()

57171    137.945408
20036    137.945408
46584    137.945408
28836    137.945408
96488    137.945408
Name: product, dtype: float64

In [52]:
largest_200_geo_2['product'].head()

35223    188.656976
40343    187.436493
49258    186.823678
99721    186.560082
5493     186.265845
Name: product, dtype: float64

взяв 500 случайных скважин, мы отобрали 200 из них с самым высоким дебитом 

In [53]:
income = 450000

In [54]:
budget = 10000000000

## расчёт прибыли по выбранным скважинам и предсказаниям модели

рассчитаем доход в каждом регионе 

In [55]:
sum_largest_200_geo_0 = largest_200_geo_0['product'].sum()
sum_largest_200_geo_0

27314.047369977838

In [56]:
sum_largest_200_geo_1 = largest_200_geo_1['product'].sum()
sum_largest_200_geo_1

23361.427999921274

In [57]:
sum_largest_200_geo_2 = largest_200_geo_2['product'].sum()
sum_largest_200_geo_2

27209.254437672535

видим, что во всех регионах проекты окупятся, но стоит отметить, что в регионе 2 прибыль будет минимальна. рассчитаем прибыль в каждом регионе 

In [58]:
income_0 = (sum_largest_200_geo_0 * 450000 - 10000000000)/1000000
income_0 

2291.3213164900267

In [59]:
income_1 = (sum_largest_200_geo_1 * 450000 - 10000000000)/1000000
income_1 

512.6425999645729

In [60]:
income_2 = (sum_largest_200_geo_2 * 450000 - 10000000000)/1000000
income_2 

2244.1644969526405

самый наименьший доход будет в регионе 2( 316 млн рублей), самый большой доход в регионе 3 ( 2,43 млрд рублей)

## Расчёт прибыли и рисков 

In [61]:
mln = 1000000

In [62]:
wells_500 = 500
top_200 = 200

In [63]:
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    revenue = selected * income 
    return sum(revenue) - budget

In [64]:
predicted_valid_0 = pd.Series(predicted_valid_0, index=target_0_valid.index)

In [65]:
state = np.random.RandomState(12345)
predicted_valid_0 = pd.Series(predicted_valid_0, index=target_0_valid.index)
values_0 = []
for i in range(1000):
    target_subsample = target_0_valid.sample(n=wells_500, replace=True, random_state=state)
    probs_subsample = predicted_valid_0[target_subsample.index]
    values_0.append(revenue(target_subsample,probs_subsample,top_200 ))
val_0 = pd.Series(values_0)

print('Cредняя выручка', val_0.mean()/mln, 'млн')
confidence_interval = st.t.interval(0.95, len(val_0)-1, val_0.mean(), val_0.sem()) 
print("95%-ый доверительный интервал:", confidence_interval)
print('2,5% квантиль', val_0.quantile(0.025)/mln, 'млн')
print('97,5% квантиль', val_0.quantile(0.975)/mln, 'млн')
print('Вероятность убытка ',val_0.apply(lambda x: x<0).sum()*100/len(val_0),'%')

Cредняя выручка 409.42803862143575 млн
95%-ый доверительный интервал: (392436312.2689623, 426419764.9739092)
2,5% квантиль -131.53602870166557 млн
97,5% квантиль 944.3955827546799 млн
Вероятность убытка  7.1 %


In [66]:
predicted_valid_1 = pd.Series(predicted_valid_1, index=target_1_valid.index)
values_1 = []
for i in range(1000):
    target_subsample = target_1_valid.sample(n=wells_500, replace=True, random_state=state)
    probs_subsample = predicted_valid_1[target_subsample.index]
    values_1.append(revenue(target_subsample,probs_subsample,top_200))
val_1 = pd.Series(values_1)
print('Cредняя выручка', val_1.mean()/mln, 'млн')
confidence_interval = st.t.interval(0.95, len(val_1)-1, val_1.mean(), val_1.sem()) 
print("95%-ый доверительный интервал:", confidence_interval)
print('2,5% квантиль', val_1.quantile(0.025)/mln, 'млн')
print('97,5% квантиль', val_1.quantile(0.975)/mln, 'млн')
print('Вероятность убытка ',val_1.apply(lambda x: x<0).sum()*100/len(val_1),'%')

Cредняя выручка 536.400199435078 млн
95%-ый доверительный интервал: (522878101.0595963, 549922297.8105597)
2,5% квантиль 112.95424712368228 млн
97,5% квантиль 998.5041566468612 млн
Вероятность убытка  0.3 %


In [67]:
predicted_valid_2 = pd.Series(predicted_valid_2, index=target_2_valid.index)
values_2 = []
for i in range(1000):
    target_subsample = target_2_valid.sample(n=wells_500, replace=True, random_state=state)
    probs_subsample = predicted_valid_2[target_subsample.index]
    values_2.append(revenue(target_subsample,probs_subsample,top_200))
val_2 = pd.Series(values_2)
print('Cредняя выручка', val_2.mean()/mln, 'млн')
confidence_interval = st.t.interval(0.95, len(val_2)-1, val_2.mean(), val_2.sem()) 
print("95%-ый доверительный интервал:", confidence_interval)
print('2,5% квантиль', val_2.quantile(0.025)/mln, 'млн')
print('97,5% квантиль', val_2.quantile(0.975)/mln, 'млн')
print('Вероятность убытка ',val_2.apply(lambda x: x<0).sum()*100/len(val_2),'%')

Cредняя выручка 339.4780341978001 млн
95%-ый доверительный интервал: (322079442.0623012, 356876626.333299)
2,5% квантиль -224.08922174407388 млн
97,5% квантиль 847.067587686391 млн
Вероятность убытка  11.8 %


Наиболее перспективный регион-второй, так как средняя прибыль в этом регионе выше, а вероятность убытков менее 1%, наименее перспективный регион-третий, имеет самую низкую среднюю прибыль и самую высокую вероятность убытков 